In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
#downloading and extracting dataset images
!unzip /content/drive/MyDrive/Applied_AI_project/dataset_2.zip

Streaming output truncated to the last 5000 lines.
  inflating: dataset_2/late blight/LB_ (264).png  
  inflating: __MACOSX/dataset_2/late blight/._LB_ (264).png  
  inflating: dataset_2/late blight/e980207c-6e67-4caf-8973-7128ca2a1c12___GHLB2 Leaf 8876.JPG  
  inflating: __MACOSX/dataset_2/late blight/._e980207c-6e67-4caf-8973-7128ca2a1c12___GHLB2 Leaf 8876.JPG  
  inflating: dataset_2/late blight/953f355a-fd2c-433e-8292-a8c1908e000e___GHLB2 Leaf 8853.JPG  
  inflating: __MACOSX/dataset_2/late blight/._953f355a-fd2c-433e-8292-a8c1908e000e___GHLB2 Leaf 8853.JPG  
  inflating: dataset_2/late blight/tomato_D33a-Late-blight-015-OMAFHRT_zoom_jpg.rf.054f2a0a7ad993f4cf6e4372933cd5bc.jpg  
  inflating: __MACOSX/dataset_2/late blight/._tomato_D33a-Late-blight-015-OMAFHRT_zoom_jpg.rf.054f2a0a7ad993f4cf6e4372933cd5bc.jpg  
  inflating: dataset_2/late blight/0b52ebed-e711-41f2-bfd1-af73410b370e___RS_Late.B 5463_flipLR.JPG  
  inflating: __MACOSX/dataset_2/late blight/._0b52ebed-e711-41f2-bfd1-af7

In [8]:
import torch
from torch.nn import CrossEntropyLoss
from torch.optim import Adam

import torchvision
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18

In [9]:
# splitting the dataset into specific folders for train, validate and test
!pip install split-folders

import splitfolders

source_folder = '/content/dataset_2'
target_folder = '/content/dataset_2_splitted'

train_ratio = .7
val_ratio = .15
test_ratio = .15

splitfolders.ratio(source_folder, output=target_folder, seed=1337, ratio=(train_ratio, val_ratio, test_ratio), group_prefix=None, move=False)

Copying files: 14658 files [00:07, 1932.15 files/s]


In [10]:
# applying data prepocessing and loading dataset
dataset_tranform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = ImageFolder("/content/dataset_2_splitted/train", transform=dataset_tranform)
validate_dataset = ImageFolder("/content/dataset_2_splitted/val", transform=dataset_tranform)
test_dataset = ImageFolder("/content/dataset_2_splitted/test", transform=dataset_tranform)

dataset_classes = train_dataset.classes
print(dataset_classes)

['early blight', 'healthy', 'late blight', 'mosaic virus', 'powdery mildew', 'septoria']


In [11]:
batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validate_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)